In [5]:
from lxml import etree
import itertools

book=etree.parse('src/book.6.xml',etree.XMLParser(remove_blank_text=True)).getroot() 
flatBook=list(book.iter())
# hand-tuned toc, do not overwrite
toc=etree.parse('src/toc.xml',etree.XMLParser()).getroot() 

def _one_only(pp):
    assert(len(pp)==1)
    return pp[0]

for toc_chap in toc:
    chap_num=toc_chap.attrib['num']
    print(chap_num)
    chap=_one_only(book.findall('.//heading-2-chapter[@toc_num="'+chap_num+'"]')).getparent()
    chapLastPara=str(max([int(e.attrib['num']) for e in chap if e.tag=='p' and 'num' in e.attrib]))
    toc_chap.attrib['para_last']=chapLastPara
    #print(chapLastPara)
    ee0={}
    # find first DOM element for each sectioning piece
    for toc_sect in toc_chap.iter():
        if toc_sect.tag=='chapter': continue
        try: para,title,starts_at=toc_sect.attrib['para'],toc_sect.attrib['title'],toc_sect.attrib.get('starts_at',None)
        except KeyError:
            print(f'Missing keyword? line {toc_sect.sourceline} {toc_sect.tag}')
            raise
        elem0=_one_only(chap.findall('.//p[@num="'+para+'"]'))
        if starts_at is not None:
            for ix in itertools.count(flatBook.index(elem0)+1):
                e=flatBook[ix]
                if e.tag not in ('span','em'): continue
                if not e.text.startswith(starts_at): continue
                assert e.getparent().tag=='p'
                elem0=e.getparent()
                break
        ee0[toc_sect]=elem0
        assert elem0.getparent().tag=='struct-2-chapter'
    def _subdivide(sects,level,lastPara):
        sect=None
        for isect,sect in enumerate(sects):
            # print(sect.sourceline)
            l2=level+3
            new=etree.Element(f'struct-{l2}-{level*"sub"}section')
            new.append(etree.Element(f'heading-{l2}-{level*"sub"}section'))
            new[-1].text=sect.attrib['title']
            new.attrib['par_begin']=sect.attrib['para']
            if isect==len(sects)-1:
                eAfter=None
                new.attrib['par_end']=sect.attrib['para_end']=lastPara
            else:
                sAfter=sects[isect+1]
                eAfter=ee0[sAfter]
                new.attrib['par_end']=sect.attrib['para_end']=str(int(sAfter.attrib['para'])-(0 if 'starts_at' in sAfter.attrib else 1))
            e=ee0[sect]
            e.addprevious(new)
            while True:
                ee=e.getnext()
                new.append(e)
                e=ee
                if e==eAfter or e is None: break
            _subdivide(sect,level=level+1,lastPara=new.attrib['par_end'])
        return [sect]
    _subdivide(toc_chap,level=0,lastPara=chapLastPara)
            

# re-apply to the book
#open('xml/book.7.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))
open('build/book.sectioned.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))
open('build/toc.more.xml','w').write(etree.tostring(toc,encoding='unicode',pretty_print=True))

I
II
III
IV
V
VI
VII
VIII
IX
X
XI
XII
XIII
XIV
XV
XVI
XVII
XVIII
XIX
XX
XXI
XXII
XXIII


21944

In [6]:
import os
##
## LaTeX output
##
book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser()).getroot() 
index=tree=etree.parse('src/index.xml',etree.XMLParser()).getroot() 
gloss=tree=etree.parse('src/gloss.xml',etree.XMLParser()).getroot() 

def _latex_writer(e,lev=0,ord=-1):
    def _rep(t): return t.replace('&','\\&')
    def _recurse(e,lev=lev):
        if e.text is not None and len(e)==0: return _rep(e.text)
        return ''.join([_latex_writer(e2,lev=lev+1,ord=ord) for ord,e2 in enumerate(e)])
    def _nobraces(t):
        #if t.strip().endswith(']'):
        return t.replace('[','').replace(']','')
        #return t
    def _title(sect,e):
        t=_recurse(e)
        if sect in ('chapter','part'): return f'\\{sect}'+'{'+t+'}'
        struct=e.getparent()
        if 'par_begin' in struct.attrib:
            p0,p1=int(struct.attrib['par_begin']),int(struct.attrib['par_end'])
            if p0==p1: t2=f'§{p0}'
            else: t2=f'§{p0}–{p1}'
            # return '\\'+sect+'[\\protect\\numberline{'+t2+'}'+t+']{\\ifplastex\\else\\kern-1em\\fi{}'+t+'}'
            return '\\'+sect+'[\\vismAlignedParas{'+t2+'}'+t+']{'+t+'}' #'\\ifplastex\\else\\hfill '+t2+'\\fi}'
            # return '\\def\\vismNextParRange{'+t2+'}\\'+sect+'{'+t+'}'
        else: return '\\'+sect+'{'+t+'}'
    ret=''
    ind=2*lev*'  '
    if e.tag=='em':
        if len(e)>0: raise RuntimeError(f'<em> with child elements, line {e.sourceline}')
        if e.text is None: return ''
        return '\\emph{'+_rep(e.text)+'}'
    elif e.tag=='span':
        if len(e)>0: raise RuntimeError(f'<span> with child elements, line {e.sourceline}')
        if e.text is None: return ''
        tx=_rep(e.text)
        if (fam:=e.attrib.get('family',None)) is None: return tx
        elif fam=='italic': return '\\emph{'+tx+'}'
        elif fam=='bold': return '\\textbf{'+tx+'}'
        elif fam=='smallcaps': return '\\textsc{'+tx+'}'
        elif fam=='bold-italic': return '\\textbf{\\emph{'+tx+'}}'
        else: raise RuntimeError(f'Unrecognized family {fam}')
    elif e.tag=='p':
        ret='\n\n'+ind if ord>0 else ''
        if 'anchor' in e.attrib:
            num,anchor=e.attrib['num'],e.attrib['anchor']
            # ret+='\\paragraph{§'+num+'.}\\vismHypertarget{'+anchor+'}{}\\marginnote{\\footnotesize\\textcolor{purple}{'+anchor+'}}{}\n'+ind
            ret+='\\vismParagraph{'+anchor+'}{'+num+'}{}\n'+ind
        return ret+_recurse(e)
    elif e.tag=='footnote':
        check=r'\vismAssertFootnoteCounter{'+e.attrib['mark']+'}'
        if 'reference_existing_footnote' in e.attrib: return check+r'\footnotemark[\value{footnote}]'
        elif anchor:=e.attrib.get('anchor',None): return '\\footnote{'+check+'\\vismHypertarget{'+anchor+'}{}\\marginnote{\\footnotesize\\textcolor{purple}{'+anchor+'}}'+_recurse(e)+'}'
        else: return '\\footnote{'+check+_recurse(e)+'}'
    elif e.tag=='verse':
        assert e[-1].tag=='line'
        e[-1].attrib['last-line']="1"
        return '\n'+ind+'\\begin{verse}\n'+_recurse(e)+ind+'\\end{verse}\n'
    elif e.tag=='line': return ind+_recurse(e)+(r'\\{}' if not 'last-line' in e.attrib else '')+'\n'
    elif e.tag=='heading-1-part':
        if e.attrib['toc_name']=='(Front)': return '' # ind+'\\frontmatter\n\n'
        label=e.attrib.get('toc_num',None)
        if label: label='part-'+label
        # label=(f'.. part-{toc_num}' if () else '')
        return '\n'+ind+_title('part',e)+(('\\label{'+label+'}'+ind+'\\vismHypertarget{'+label+'}\n') if label else '')
    # '\\part{'+_recurse(e)+'}\n'
    elif e.tag=='heading-2-chapter':
        # \label is just for PlasTeX which can then name the output file accordingly (the chapter has an $id)
        if toc_num:=e.attrib.get('toc_num',None): return '\n'+ind+_title('chapter',e)+('\\label{'+toc_num+'}')+ind+'\\vismHypertarget{'+toc_num+'}\n'
        else: return '\n'+ind+_title('chapter',e)
    elif e.tag=='heading-3-section': return '\n'+ind+_title('section',e)
    elif e.tag=='heading-4-subsection': return '\n'+ind+_title('subsection',e)
    elif e.tag=='heading-5-subsubsection': return '\n'+ind+_title('subsubsection',e)
    elif e.tag in ('struct-2-chapter','struct-3-section','struct-4-subsection','struct-5-subsubsection'): return _recurse(e)
    elif e.tag=='struct-1-part':
        if e.attrib['name']=='Part I':
            pre,post=ind+r'\mainmatter',''
        elif e.attrib['name']=='Part III': pre,post='',ind+'\\appendix'
        else: pre,post='',''
        return pre+_recurse(e)+post
    elif e.tag=='printed_page':
        #if e.attrib['edition']=='BPS2011': return r'{\small\textbf{\href[page='+e.attrib['page_id']+']{PathofPurification2011.pdf}{\{'+e.text+' ('+e.attrib['page_id']+')\}}}}' # marginpar{['+e.text+r']}'
        if e.attrib['edition']=='BPS2011':
            # return r'\marginnote[\footnotesize\{'+e.text+'('+e.attrib['page_id']+r')\}]{}[-1ex]' # this is too complicated for PlasTeX
            return r'\marginnote{\textcolor{teal}{\footnotesize\{'+e.text+'('+e.attrib['page_id']+r')\}}}{}'
        elif e.attrib['edition']=='PTS': return r'\textcolor{brown}{\textit{['+e.text+']}}'
        assert False
    elif e.tag=='ref':
        if e.attrib['type']=='vism': return r'\hyperlink{'+e.attrib['target']+r'}{'+e.text+'}{}'
        elif e.attrib['type']=='bib': return r'\textbf{\cite{'+e.attrib['target']+'}'+(e.attrib['loc'] if 'loc' in e.attrib else '')+'}'
        # r'\fbox{'+e.text+'→'+e.attrib['target']+'}'
        assert False
    elif e.tag in ('index','glossary'):
        title,subtitle=_rep(e.attrib['title']),_rep(e.attrib.get('subtitle',None))
        ret='\\chapter['+title+']{'+title+'\\* {\large '+subtitle+'}}'
        if ii:=e.findall('introductory'):
            assert len(ii)==1
            ret+=_recurse(ii[0])
        ret+=r'\begin{multicols}{2}\parskip=.2\baselineskip\RaggedRight\parindent=-1em\leftskip=1em '+_recurse(e)+r'\end{multicols}'
        return ret
    elif e.tag=='introductory': return '' # already handled in index/glossary
    elif e.tag=='entry':
        return r'\par\textbf{'+e.attrib['title']+'} '+_recurse(e)+'\n'
    elif e.tag=='raw':
        return open('latex/'+e.attrib['file']+'.tex','r').read()
    elif e.tag=='TODO':
        return r'\textbf{[TODO: '+e.text+']}'
    raise RuntimeError(f'Unhandled tag <{e.tag}>')

os.makedirs('build/latex',exist_ok=True)
print('BODY')
open('build/latex/vism-body.tex','w').write(''.join(_latex_writer(e) for e in book))
print('INDEX')
open('build/latex/vism-index.tex','w').write(''.join(_latex_writer(index)))
print('GLOSSARY')
open('build/latex/vism-glossary.tex','w').write(''.join(_latex_writer(gloss)))

    

BODY
INDEX
GLOSSARY


56753

In [7]:
from lxml import etree
##
## Sphinx
##
book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser()).getroot() 
index=tree=etree.parse('src/index.xml',etree.XMLParser()).getroot() 
gloss=tree=etree.parse('src/gloss.xml',etree.XMLParser()).getroot() 


class SphinxWriter(object):
    def __init__(self,outdir):
        self.footnotes={}
        self.outdir=outdir
        self.chapter=0
        self.part=0
    def _flush(self):
        if not self.footnotes: return ''
        ret='\n\n.. rubric:: Footnotes\n\n'
        # TODO: multi-paragraph footnotes
        for k,vv in self.footnotes.items(): ret+=f'\n\n.. _{self.chapter_anchor}.n{k}:\n\n.. [#{k}] '+'\n    '.join([v for v in vv.split('\n')])+'\n'
        self.footnotes={}
        return ret
    def _rep(self,t): return t # .replace('&','\\&') 
    def recurse(self,e):
        if e.text is not None and len(e)==0: return self._rep(e.text)
        return ''.join([self.write(e2,ord=ord) for ord,e2 in enumerate(e)])
    def title(self,e,level,anchor=None,prefix=None):
        ret=''
        if anchor: ret+='\n\n.. _'+anchor+':'
        t=(e if isinstance(e,str) else self.recurse(e))
        if prefix: t=prefix+'. '+t
        return ret+'\n\n'+t+'\n'+len(t)*('#*=-^"'[level])
    def enclose(self,t,c):
        if t.strip()=='': return ' '
        ret=t
        if ret.endswith(' '): ret=ret.rstrip()+c+' '
        else: ret=ret+c+'\\ '
        if ret.startswith(' '): ret=' '+c+ret.lstrip()
        else: ret=c+ret
        return ret

    def write(self,e,ord=-1):
        def _nobraces(t):
            #if t.strip().endswith(']'):
            return t.replace('[','').replace(']','')
            #return t
        if e.tag=='em':
            assert len(e)==0
            if e.text is None: return ''
            return self.enclose(self._rep(e.text),'*')
        elif e.tag=='span':
            assert len(e)==0
            if e.text is None: return ''
            tx=self._rep(e.text)
            if (fam:=e.attrib.get('family',None)) is None: return tx
            elif fam=='italic': return self.enclose(tx,'*')
            elif fam=='bold': return self.enclose(tx,'**')
            elif fam=='smallcaps': return self.enclose(tx,'``')
            elif fam=='bold-italic': return self.enclose(tx,'``')
            else: raise RuntimeError(f'Unrecognized family {fam}')
        elif e.tag=='p':
            if anchor:=e.attrib.get('anchor',None): pre=f'\n\n.. _{anchor}:\n\n**§{e.attrib["num"]}** '
            else: pre=('\n\n' if ord>0 else '')
            return pre+self.recurse(e)
        # elif e.tag=='vism-para': return f'\n\n.. _{self.fixanchor(e.attrib["anchor"])}:\n\n**§{e.text}** '
        elif e.tag=='footnote':
            anchor=e.attrib.get("anchor",str(len(self.footnotes)+1))
            mark=e.attrib['mark']
            if 'reference_existing_footnote' in e.attrib:
                return f' [#{mark}]_'
            self.footnotes[mark]=self.recurse(e)
            return f' [#{mark}]_ '
        elif e.tag=='verse':
            return '\n\n'+self.recurse(e)
        elif e.tag=='line': return ('\n\n' if ord==0 else '')+'\n| '+self.recurse(e)+('\n' if 'last-line' in e.attrib else '')
        elif e.tag=='heading-1-part':
            anchor=e.attrib.get('toc_num',None)
            if anchor: anchor='part-'+anchor
            self.partOut.write(self.title(e,level=1,anchor=anchor,prefix=e.attrib["toc_name"])+'\n\n.. toctree::\n   :numbered:\n   :maxdepth: 6\n\n')
            return ''
        elif e.tag=='heading-2-chapter':
            self.chapter_anchor=e.attrib.get('toc_num',None)
            return self.title(e,level=2,anchor=e.attrib.get('toc_num',None),prefix=e.attrib.get('toc_num',None))
        elif e.tag=='heading-3-section':
            return self.title(e,level=3)
        elif e.tag=='heading-4-subsection':
            return self.title(e,level=4)
        elif e.tag=='heading-5-subsubsection':
            return self.title(e,level=5)
        elif e.tag in ('struct-3-section','struct-4-subsection','struct-5-subsubsection'): return self.recurse(e)
        elif e.tag=='struct-1-part':
            self.part+=1
            f=f'{self.outdir}/part-{self.part}.rst'
            self.partOut=open(f,'w')
            print(f'→ {f}')
            for chap in e:
                self.partOut.write('\n   '+self.write(chap))
            self.partOut.close()
            return None
        elif e.tag=='struct-2-chapter':
            self.chapter+=1
            f=f'ch-{self.chapter:02d}.rst'
            ff=f'{self.outdir}/{f}'
            print(f'   → {ff}')
            out=open(ff,'w')
            out.write(self.recurse(e)+self._flush())
            return f
        elif e.tag=='book':
            for e2 in e: self.write(e2)
            return None
        # elif e.tag=='footref': return f'[#{e.text}]_'
        elif e.tag=='printed_page':
            #if e.attrib['edition']=='BPS2011': return r'{\small\textbf{\href[page='+e.attrib['page_id']+']{PathofPurification2011.pdf}{\{'+e.text+' ('+e.attrib['page_id']+')\}}}}' # marginpar{['+e.text+r']}'
            if e.attrib['edition']=='BPS2011': return f'*[{e.text}/{e.attrib["page_id"]}]* '
            elif e.attrib['edition']=='PTS': return f' ``{e.text}`` '
            assert False
        elif e.tag=='ref':
            if e.attrib['type']=='vism':
                return f':ref:`{e.text} <{e.attrib["target"]}>`'
            elif e.attrib['type']=='bib': return f' [{e.attrib["target"]}]_ '+(self.enclose(e.attrib["loc"],'*') if 'loc' in e.attrib else '')+' '
            assert False
        elif e.tag in ('index','glossary'):
            title,subtitle=e.attrib['title'],e.attrib['subtitle']
            if e.tag=='index':      out,ret='index_',self.title(e=f'{title} ({subtitle})',level=1,anchor='index')
            elif e.tag=='glossary': out,ret='glossary',self.title(e=f'{title} ({subtitle})',level=1,anchor='glossary')
            if ii:=e.findall('introductory'):
                assert len(ii)==1
                ret+='\n\n'+self.recurse(ii[0])
            ret+='\n\n.. glossary::'
            ret+=self.recurse(e)
            f=f'{self.outdir}/{out}.rst'
            print(f'→ {f}')
            open(f,'w').write(ret)
            return
        elif e.tag=='introductory': return ''
        elif e.tag=='entry':
            title=e.attrib["title"].replace("*","\\*")
            return f'\n\n   {title}\n          '+self.recurse(e) # {e.attrib["desc"]}'
        elif e.tag=='raw':
            return '\n\n'+open('sphinx/'+e.attrib['file']+'.rst','r').read()+'\n\n'
        elif e.tag=='TODO':
            return '**TODO: {e.text}**\ '
        raise RuntimeError(f'Unhandled tag <{e.tag}>')
os.makedirs('build/sphinx/source',exist_ok=True)
writer=SphinxWriter(outdir='build/sphinx/source')
writer.write(book)
writer.write(index)
writer.write(gloss)
# ''.join(_latex_writer(e) for e in book))
#open('/tmp/vism-index.tex','w').write(''.join(_latex_writer(index,list='index')))
#open('/tmp/vism-glossary.tex','w').write(''.join(_latex_writer(gloss,list='glossary')))

    

→ build/sphinx/source/part-1.rst
   → build/sphinx/source/ch-01.rst
   → build/sphinx/source/ch-02.rst
→ build/sphinx/source/part-2.rst
   → build/sphinx/source/ch-03.rst
   → build/sphinx/source/ch-04.rst
→ build/sphinx/source/part-3.rst
   → build/sphinx/source/ch-05.rst
   → build/sphinx/source/ch-06.rst
   → build/sphinx/source/ch-07.rst
   → build/sphinx/source/ch-08.rst
   → build/sphinx/source/ch-09.rst
   → build/sphinx/source/ch-10.rst
   → build/sphinx/source/ch-11.rst
   → build/sphinx/source/ch-12.rst
   → build/sphinx/source/ch-13.rst
   → build/sphinx/source/ch-14.rst
   → build/sphinx/source/ch-15.rst
→ build/sphinx/source/part-4.rst
   → build/sphinx/source/ch-16.rst
   → build/sphinx/source/ch-17.rst
   → build/sphinx/source/ch-18.rst
   → build/sphinx/source/ch-19.rst
   → build/sphinx/source/ch-20.rst
   → build/sphinx/source/ch-21.rst
   → build/sphinx/source/ch-22.rst
   → build/sphinx/source/ch-23.rst
   → build/sphinx/source/ch-24.rst
   → build/sphinx/source/ch-

In [8]:
## DocBook
from lxml import etree
import os

book=tree=etree.parse('build/book.sectioned.xml',etree.XMLParser(remove_blank_text=True)).getroot()
index=tree=etree.parse('src/index.xml',etree.XMLParser(remove_blank_text=True)).getroot() 
gloss=tree=etree.parse('src/gloss.xml',etree.XMLParser(remove_blank_text=True)).getroot()
book.append(index)
book.append(gloss)
bibDocbook=etree.parse('docbook/bib.xml',etree.XMLParser(remove_blank_text=True)).getroot()

frontmatter=False

def _docb_writer(e,ord=0,parent=None,xslTNG=False):
    xlinkNs='http://www.w3.org/1999/xlink'
    xmlNs='http://www.w3.org/XML/1998/namespace'
    xmlPrefix='{'+xmlNs+'}'
    
    if xslTNG:
        xlinkPrefix='{'+xlinkNs+'}'
        nsmap={None:'http://docbook.org/ns/docbook','xlink':xlinkNs,'xml':xmlNs}
    else:
        #nsmap={None:'http://docbook.org/ns/docbook','xml':xmlNs}
        #xlinkPrefix=''
        nsmap,xlinkPrefix=None,''

    def _E(e,text=None,*,subs=[],xml_id=None,**kw):
        assert text is None or isinstance(text,str)
        ret=etree.Element(e,**kw,nsmap=nsmap) # ,'pub':pubNs})
        if xml_id is not None:
            # if xslTNG:
            ret.attrib[xlinkPrefix+'label']=xml_id
            ret.attrib[xmlPrefix+'id']=xml_id
        if 'linkend' in kw:
            if xslTNG:
                ret.attrib[xlinkPrefix+'to']=kw['linkend']
                kw.pop('linkend')
            pass
        ret.text=text
        for sub in subs:
            if sub is None: continue
            if sub.tag=='__FLATTEN__':
                for su in sub: ret.append(su)
            else: ret.append(sub)
        return ret

    def _curr_chapter(e):
        while (e:=e.getparent()) is not None:
            if e.tag=='struct-2-chapter':
                heading=e.findall('heading-2-chapter')[0]
                return heading.attrib.get('toc_num',None)


    def _recurse(e,dbg=False):
        if e.text is not None and len(e)==0:
            if dbg: print('=',e.text)
            return [_E('phrase',e.text)]
        if e.text is not None and e.text!='' and len(e)>0:
            print('$$$$',e.tag,e.sourceline) # ,len(e.text),e.text)
        ret=[_docb_writer(e2,ord=ord,parent=e) for ord,e2 in enumerate(e)]
        if dbg: print('|'.join([r.text for r in ret if r.text]))
        return ret
    if e.tag=='book':
        # return _E('book',xmlns='http://docbook.org/ns/docbook',version="5.0",subs=_recurse(e))
        b=_E('book',version="5.2",subs=_recurse(e))
        b.append(bibDocbook)
        return b
        # b.attrib['xmlns:pub']="http://docbook.org/ns/docbook/publishers"
    if e.tag=='em':
        assert len(e)==0
        return _E('emphasis',e.text)  
    elif e.tag=='span':
        assert len(e)==0
        if e.text is None: return None
        if (fam:=e.attrib.get('family',None)) is None: return _E('phrase',e.text)
        elif fam=='italic': return _E('emphasis',e.text)
        elif fam=='bold': return _E('emphasis',e.text,role='bold')
        elif fam=='smallcaps': return _E('emphasis',e.text,role='smallcaps')
        elif fam=='bold-italic': return _E('emphasis',e.text,role='bold-italic')
        else: raise RuntimeError(f'Unrecognized family {fam}')
    elif e.tag=='p':
        if 'anchor' in e.attrib:
            return _E('formalpara',xml_id=e.attrib['anchor'],subs=[_E('title','§'+e.attrib['num']),_E('para',subs=_recurse(e))])
            # add this paragraph and all next <p> and <verse> up to numbered p or end of section
            #subs=[_E('title','§'+e.attrib['num']),e]
            #sub=e
            #while ((sub:=sub.getnext()) is not None) and ((sub.tag=='verse') or (sub.tag=='p' and 'anchor' not in sub.attrib)):
            #    subs+=_recurse(sub)
            #return _E('formalpara',xml_id=e.attrib['anchor'],subs=subs)            
        else: return _E('para',subs=_recurse(e))
    elif e.tag=='footnote':
        if (ch:=_curr_chapter(e)) is None:
            assert 'reference_existing_footnote' not in e.attrib
            return _E('footnote',subs=_recurse(e))
        else:
            label=f'{ch}.n{e.attrib["mark"]}'
            if 'reference_existing_footnote' in e.attrib: return _E('footnoteref',linkend=label)
            return _E('footnote',subs=_recurse(e),xml_id=label)
    elif e.tag=='verse':
        for ich,ch in enumerate(e):
            if not ch.tag=='line':
                raise RuntimeError(f'<verse> may contain only <line> elements (<verse> at line {e.sourceline}; child #{ich} <{ch.tag}>, line {ch.sourceline})')
        return _E('linegroup',subs=[_E('speaker')]+_recurse(e))
        return _E('linegroup',subs=[_E('speaker')]+_recurse(e))
        # return _E('poetry',subs=[_E('linegroup',subs=_recurse(e))])
    elif e.tag=='line': return _E('line',subs=_recurse(e))
    elif e.tag in ('heading-1-part','heading-2-chapter','heading-3-section','heading-4-subsection','heading-5-subsubsection'):
        if e.tag=='heading-1-part' and e.attrib['toc_name']=='(Front)': return None
        return _E('title',subs=_recurse(e))
    elif e.tag=='struct-1-part':
        global frontmatter
        heading=e.findall('heading-1-part')[0]
        # flatten preface
        if (toc_num:=heading.attrib.get('toc_num',None)) is None:
            frontmatter=True
            return _E('__FLATTEN__',subs=_recurse(e)) # _E('preface',subs=_recurse(e))
        frontmatter=False
        return _E('part',subs=_recurse(e),xml_id='part-'+toc_num)
    elif e.tag=='struct-2-chapter':
        heading=e.findall('heading-2-chapter')[0]
        if frontmatter: return _E('preface',subs=_recurse(e))
        if 'toc_num' in heading.attrib:  return _E('chapter',subs=_recurse(e),xml_id=heading.attrib['toc_num'])
        else: return _E('section',subs=_recurse(e))
    elif e.tag in ('struct-3-section','struct-4-subsection','struct-5-subsubsection'): return _E('section',subs=_recurse(e))
    elif e.tag=='printed_page':
        # return None # XXXXX
        if e.attrib['edition']=='BPS2011': return _E('literal',f'[{e.text}|{e.attrib["page_id"]}]')
        elif e.attrib['edition']=='PTS': return _E('varname',f'({e.text})')
    elif e.tag=='ref':
        if e.attrib['type']=='vism': return _E('link',e.text,linkend=e.attrib['target'])
        elif e.attrib['type']=='bib':
            if 'loc' in e.attrib: return _E('phrase',subs=[_E('citation',e.attrib['target']),_E('phrase',e.attrib['loc'])])
            return _E('citation',e.attrib['target'])
        assert False
    elif e.tag=='index': return _E('index',subs=[_E('title',e.attrib['title']),_E('subtitle',e.attrib['subtitle'])]+_recurse(e))
    elif e.tag=='glossary':  return _E('glossary',subs=[_E('title',e.attrib['title']),_E('subtitle',e.attrib['subtitle'])]+_recurse(e))
    elif e.tag=='introductory': return _E('para',subs=_recurse(e))
    elif e.tag=='entry':
        if parent.tag=='index': return _E('primaryie',subs=[_E('emphasis',text=e.attrib['title'],role='bold')]+_recurse(e))
        elif parent.tag=='glossary': return _E('glossentry',subs=[_E('glossterm',text=e.attrib['title'])]+[_E('glossdef',subs=_recurse(e))])
        print(parent.tag)
        assert False
    elif e.tag=='raw':
        return etree.parse('docbook/'+e.attrib['file']+'.xml',etree.XMLParser(remove_blank_text=True)).getroot()
    elif e.tag=='TODO':
        return _E('emphasis','[TODO: '+e.text+']',role='bold')
    raise RuntimeError(f'Unhandled tag <{e.tag}>')

def _fix_formalpara(book):
    for fp in book.findall('.//formalpara'):
        subs=[]
        p=fp
        while ((p:=p.getnext()) is not None) and (p.tag in ('para','linegroup')):
            subs.append(p)
        for p in subs: fp.append(p)
    return book
            
for xslTNG in True,False:
    docb=_docb_writer(book,xslTNG=xslTNG)
    docb=_fix_formalpara(docb)
    kw=dict(doctype=None,xml_declaration=True,encoding='utf-8')
    os.makedirs('build/docbook',exist_ok=True)
    stem='vism.xslTNG' if xslTNG else 'vism'
    open(f'build/docbook/{stem}.xml','wb').write(etree.tostring(docb,pretty_print=False,**kw))
    open(f'build/docbook/{stem}.pretty.xml','wb').write(etree.tostring(docb,pretty_print=True,**kw))
    print(f'→ build/docbook/{stem}.xml build/docbook/{stem}.pretty.xml')

→ build/docbook/vism.xslTNG.xml build/docbook/vism.xslTNG.pretty.xml
→ build/docbook/vism.xml build/docbook/vism.pretty.xml
